<a href="https://colab.research.google.com/github/v8karlo/OpenHantek6022/blob/master/ISR_simplified(youtube).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>The goal of this project is to upscale and improve the quality of low resolution images.</h2>
<p>Docs: <a href = "https://idealo.github.io/image-super-resolution/">https://idealo.github.io/image-super-resolution/</a></p>
Original Notebook: <a href = "https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb">https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb</a>



In [4]:
#@title Run this cell to Setup ISR (Image Super-Resolution)

#@markdown * installs ISR
#@markdown * downloads pre-trained weights and data
#@markdown * creates the models (RRDN GANS model, Large RDN model, Small RDN model, Large RDN noise cancelling, detail enchancing model)


!pip -q install ISR gdown

#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/ArtefactCancelling/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/PSNR-driven/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C3-D10-G64-G064-x2/PSNR-driven/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rrdn-C4-D3-G32-G032-T10-x4/Perceptual/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5

import os
if not os.path.exists('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf'):
  # download pretrained weights
  import os
  github_weights_url = 'https://github.com/AwaleSajil/ISR_weights'
  folder_name = os.path.basename(github_weights_url)
  !git clone $github_weights_url
  !mv $folder_name 'weights'


!git clone https://github.com/AwaleSajil/ISR-Youtube-


from ISR.models import RDN, RRDN
import numpy as np
from PIL import Image
import IPython.display


rrdn_gans = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'x':4, 'T': 10})
rrdn_gans.model.load_weights('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5')

large_rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
large_rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')

small_rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
small_rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')

large_noise_cancelling_rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
large_noise_cancelling_rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')

  Preparing metadata (setup.py) ... done
ERROR: Cannot install isr==1.9.0, isr==1.9.1, isr==1.9.2, isr==1.9.3, isr==2.0.0, isr==2.0.1, isr==2.0.2, isr==2.0.5, isr==2.1, isr==2.1.1 and isr==2.2.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
fatal: destination path 'ISR_weights' already exists and is not an empty directory.
mv: cannot move 'ISR_weights' to 'weights/ISR_weights': Directory not empty
fatal: destination path 'ISR-Youtube-' already exists and is not an empty directory.


ModuleNotFoundError: No module named 'ISR'

In [ ]:
#@title Choose a model, specific a picture and Run to get RESULTS

#@markdown * Choose a model
choice_model = 'RDN noise cancelling (C6-D20-G64-G064-x2_ArtefactCancelling_epoch219)' #@param ["RRDN GANS (C4-D3-G32-G032-T10-x4_epoch299)", "Large RDN (C6-D20-G64-G064-x2_PSNR_epoch086)", "Small RDN (C3-D10-G64-G064-x2_PSNR_epoch134)", "RDN noise cancelling (C6-D20-G64-G064-x2_ArtefactCancelling_epoch219)"]


#@markdown * Specify an image (from internet or upload your own)
imageURL__or__pathToImage = '' #@param {type:"string"}


sourceImg = None
def path_leaf(path):
  import ntpath
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

try:
  fname = path_leaf(imageURL__or__pathToImage)
  sourceImg = Image.open(fname)
except FileNotFoundError:
  !curl $imageURL__or__pathToImage > 'downloaded_img.png'
  fname = 'downloaded_img.png'
  sourceImg = Image.open(fname)
except:
  raise Exception("Image address is incorrect")


if choice_model == 'RRDN GANS (C4-D3-G32-G032-T10-x4_epoch299)':
  sr_img = rrdn_gans.predict(np.array(sourceImg))

elif choice_model == 'Large RDN (C6-D20-G64-G064-x2_PSNR_epoch086)':
  sr_img = large_rdn.predict(np.array(sourceImg))

elif choice_model == 'Small RDN (C3-D10-G64-G064-x2_PSNR_epoch134)':
  sr_img = small_rdn.predict(np.array(sourceImg))

else:
  sr_img = large_noise_cancelling_rdn.predict(np.array(sourceImg))

# Image.fromarray(sr_img)


result_img = Image.fromarray(sr_img)
result_img.save("/content/result.png")

resizedOriginal = sourceImg.resize(size=(result_img.size[0], result_img.size[1]), resample=Image.BICUBIC)
resizedOriginal.save("/content/original.png")

!cp '/content/result.png' '/usr/local/share/jupyter/nbextensions/result.png'
!cp '/content/original.png' '/usr/local/share/jupyter/nbextensions/original.png'

width, height = result_img.size

import time
from IPython.display import clear_output


with open('/content/ISR-Youtube-/index.html','r') as file:
    htm = file.read()

time.sleep(1)
html_code = htm.format(width, height)
print()
print("Original Image (scaled to match output res)")
display(Image.open('/content/original.png'))
print("Result")
display(Image.open('/content/result.png'))

In [ ]:
#@title Run this cell to get COMPARE results
#@markdown Rerun this cell if slider doesnout respond (might solve)

display(IPython.display.HTML(html_code))